## Importing libraries

In [192]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import time
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

### Importing data

In [193]:
train=pd.read_csv("beg-hyp/weather_train.csv")
test=pd.read_csv("beg-hyp/weather_test.csv")

In [194]:
train.head()

,Unnamed: 0,DATE (YYYY/MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,Snow Depth [cm]
0,0,2020/02/22,04:11,-1.12724,-0.365968,77.66760,8.21,0.388,-12.412,19.20,1.20,300.10,817.297,0.000,0.0,0.0000,0,50.16,9.460
1,1,2020/07/25,20:10,-1.04030,0.093816,304.73416,20.14,14.855,11.955,59.29,4.45,262.40,820.184,5.334,0.0,0.0000,7,57.39,1.121
2,2,2020/09/13,04:03,-1.29707,-0.251820,68.24852,14.08,6.560,-0.940,35.17,2.45,280.20,822.653,0.000,0.0,0.0000,8,65.63,-0.050
3,3,2020/03/08,15:25,79.20020,-0.014836,237.11811,15.03,5.725,-4.575,24.36,3.70,23.07,809.641,0.000,0.0,0.1587,6,57.08,0.734
4,4,2020/11/17,19:33,-1.61485,-0.459201,271.20406,12.66,3.180,-10.220,17.37,2.70,283.80,818.717,0.000,0.0,0.0000,0,128.59,0.013


## data manipulation

In [195]:
train = train.rename(columns={"DATE (YYYY/MM/DD)":"date","Snow Depth [cm]":"depth"})
test = test.rename(columns={"DATE (YYYY/MM/DD)":"date","Snow Depth [cm]":"depth"})

In [196]:
train.date=pd.to_datetime(train.date)
train['year']=train["date"].dt.year
train["month"]=train["date"].dt.month
train["day"]=train["date"].dt.day
train.drop(columns=["Unnamed: 0","date"],axis=1,inplace=True)
train[['Hour', 'Minutes']] = train['MST'].str.split(':', expand=True)
# display the dataframe


In [197]:
train.drop(columns=["MST","Minutes"],axis=1,inplace=True)
train["depth"] = train["depth"]/100

In [198]:
test.date=pd.to_datetime(test.date)
test['year']=test["date"].dt.year
test["month"]=test["date"].dt.month
test["day"]=test["date"].dt.day
test.drop(columns=["Unnamed: 0","date"],axis=1,inplace=True)
test[['Hour', 'Minutes']] = test['MST'].str.split(':', expand=True)

In [199]:
test.drop(columns=["MST","Minutes"],axis=1,inplace=True)

In [200]:
train.head()

,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,depth,year,month,day,Hour
0,-1.12724,-0.365968,77.66760,8.21,0.388,-12.412,19.20,1.20,300.10,817.297,0.000,0.0,0.0000,0,50.16,0.09460,2020,2,22,04
1,-1.04030,0.093816,304.73416,20.14,14.855,11.955,59.29,4.45,262.40,820.184,5.334,0.0,0.0000,7,57.39,0.01121,2020,7,25,20
2,-1.29707,-0.251820,68.24852,14.08,6.560,-0.940,35.17,2.45,280.20,822.653,0.000,0.0,0.0000,8,65.63,-0.00050,2020,9,13,04
3,79.20020,-0.014836,237.11811,15.03,5.725,-4.575,24.36,3.70,23.07,809.641,0.000,0.0,0.1587,6,57.08,0.00734,2020,3,8,15
4,-1.61485,-0.459201,271.20406,12.66,3.180,-10.220,17.37,2.70,283.80,818.717,0.000,0.0,0.0000,0,128.59,0.00013,2020,11,17,19


In [201]:
test.head()

,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,year,month,day,Hour
0,-0.821587,-0.197822,270.39668,0.344,-3.269,-9.469,43.43,1.95,300.70,817.643,0.0,0.0,0.0000,7,163.53,2020,1,23,19
1,-0.464099,0.039564,83.18453,-15.420,-16.249,-19.649,67.07,1.20,308.30,809.469,0.0,0.0,0.0000,0,84.09,2020,5,2,04
2,-1.133330,-0.009875,35.33772,22.380,11.522,3.122,28.25,1.70,214.00,813.392,0.0,0.0,0.0000,7,109.24,2020,6,6,02
3,849.620000,946.128000,244.34737,14.240,6.585,-1.215,34.02,3.45,45.53,821.467,0.0,0.0,0.1844,7,156.80,2020,10,5,14
4,44.193400,0.488831,230.89280,-5.580,-6.169,-7.269,86.40,1.20,15.69,812.324,0.0,1.0,0.6648,7,154.89,2020,11,12,15


## visualization

In [202]:
train.head()

,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,depth,year,month,day,Hour
0,-1.12724,-0.365968,77.66760,8.21,0.388,-12.412,19.20,1.20,300.10,817.297,0.000,0.0,0.0000,0,50.16,0.09460,2020,2,22,04
1,-1.04030,0.093816,304.73416,20.14,14.855,11.955,59.29,4.45,262.40,820.184,5.334,0.0,0.0000,7,57.39,0.01121,2020,7,25,20
2,-1.29707,-0.251820,68.24852,14.08,6.560,-0.940,35.17,2.45,280.20,822.653,0.000,0.0,0.0000,8,65.63,-0.00050,2020,9,13,04
3,79.20020,-0.014836,237.11811,15.03,5.725,-4.575,24.36,3.70,23.07,809.641,0.000,0.0,0.1587,6,57.08,0.00734,2020,3,8,15
4,-1.61485,-0.459201,271.20406,12.66,3.180,-10.220,17.37,2.70,283.80,818.717,0.000,0.0,0.0000,0,128.59,0.00013,2020,11,17,19


In [203]:
test.head()

,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,year,month,day,Hour
0,-0.821587,-0.197822,270.39668,0.344,-3.269,-9.469,43.43,1.95,300.70,817.643,0.0,0.0,0.0000,7,163.53,2020,1,23,19
1,-0.464099,0.039564,83.18453,-15.420,-16.249,-19.649,67.07,1.20,308.30,809.469,0.0,0.0,0.0000,0,84.09,2020,5,2,04
2,-1.133330,-0.009875,35.33772,22.380,11.522,3.122,28.25,1.70,214.00,813.392,0.0,0.0,0.0000,7,109.24,2020,6,6,02
3,849.620000,946.128000,244.34737,14.240,6.585,-1.215,34.02,3.45,45.53,821.467,0.0,0.0,0.1844,7,156.80,2020,10,5,14
4,44.193400,0.488831,230.89280,-5.580,-6.169,-7.269,86.40,1.20,15.69,812.324,0.0,1.0,0.6648,7,154.89,2020,11,12,15


In [204]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Global CMP22 (vent/cor) [W/m^2]        15000 non-null  float64
 1   Direct sNIP [W/m^2]                    15000 non-null  float64
 2   Azimuth Angle [degrees]                15000 non-null  float64
 3   Tower Dry Bulb Temp [deg C]            15000 non-null  float64
 4   Tower Wet Bulb Temp [deg C]            15000 non-null  float64
 5   Tower Dew Point Temp [deg C]           15000 non-null  float64
 6   Tower RH [%]                           15000 non-null  float64
 7   Peak Wind Speed @ 6ft [m/s]            15000 non-null  float64
 8   Avg Wind Direction @ 6ft [deg from N]  15000 non-null  float64
 9   Station Pressure [mBar]                15000 non-null  float64
 10  Precipitation (Accumulated) [mm]       15000 non-null  float64
 11  Mo

### Feature scaling

In [205]:
features=["Global CMP22 (vent/cor) [W/m^2]","Direct sNIP [W/m^2]","Azimuth Angle [degrees]","Tower Dry Bulb Temp [deg C]","Tower Wet Bulb Temp [deg C]","Tower Dew Point Temp [deg C]","Tower RH [%]","Peak Wind Speed @ 6ft [m/s]","Avg Wind Direction @ 6ft [deg from N]","Station Pressure [mBar]","Precipitation (Accumulated) [mm]","Moisture","Albedo (CMP11)","Cloud Type","Solar Zenith Angle","year","month","day","Hour"]
target=["depth"]

In [207]:
X = train[features]
y = train[target]

In [208]:
scalar = preprocessing.StandardScaler().fit(X)
x =  scalar.transform(X)               #  feature scaling 

In [209]:
x

array([[-0.68784071, -0.648703  , -1.02316414, ..., -1.30934485,
         0.70854112, -1.09293486],
       [-0.68754143, -0.64745343,  1.23349252, ...,  0.14376056,
         1.0498532 ,  1.22387313],
       [-0.68842534, -0.64839278, -1.11677383, ...,  0.72500273,
        -0.31539512, -1.09293486],
       ...,
       [ 0.30193024,  1.72473089, -0.39402533, ...,  1.59686597,
         1.73247736, -0.36893237],
       [-0.68873994, -0.64753395, -1.72916948, ..., -0.14686052,
        -1.11178997, -1.67213686],
       [ 0.81480715,  1.43281513,  0.73671095, ...,  0.72500273,
        -1.33933135,  0.49987063]])

In [234]:
print(x[0])

[-0.68784071 -0.648703   -1.02316414 -0.35323447 -0.62308407 -1.27914199
 -0.87889739 -0.78914956  1.13125833  0.07676455 -0.19701102 -0.26302475
 -0.71772948 -0.83533362 -0.97908117  0.         -1.30934485  0.70854112
 -1.09293486]


## model making

In [212]:
x = np.asanyarray(x)   
y = np.asanyarray(y)     #numpy array

In [213]:
x_train , x_test , y_train , y_test = train_test_split(x, y, test_size = 0.20, random_state= 4 )

In [214]:
len(x_train)

12000

In [215]:
model = LinearRegression()

In [216]:
model.fit(x_train,y_train)

LinearRegression()

In [217]:
print('Coffiencient : ' , model.coef_)

Coffiencient :  [[-3.14616951e-02  3.03141766e-02  9.84105868e-02  8.78040889e-02
  -9.51858343e-02  2.56130166e-02 -4.52333784e-03 -1.89997962e-02
   3.52385102e-02  6.83461870e-05 -1.64246104e-01 -5.63486921e-02
   6.88800490e-02 -1.33865121e-02 -1.51403377e-02 -5.55111512e-17
  -3.38561327e-02  1.34319262e-02 -2.06347114e-02]]


## prediction

In [218]:
test_pred=  model.predict(x_test)

In [219]:
test_pred

array([[-0.16303769],
       [-0.07928688],
       [ 0.08637021],
       ...,
       [ 0.27419842],
       [-0.12522264],
       [ 0.12706492]])

In [220]:
print("residual sum of squares : %.2f" % np.mean((test_pred - y_test) ** 2))

residual sum of squares : 8.42


In [221]:
print('Variance score: %.2f' % model.score(x, y))

Variance score: 0.01


### Test prediction

In [222]:
# feature scaling
scalar = preprocessing.StandardScaler().fit(test)
test =  scalar.transform(test)

In [223]:
test

array([[-0.71378751, -0.66751449,  0.91881239, ..., -1.47577524,
         0.82897473,  1.12491671],
       [-0.71260561, -0.66688834, -0.99972505, ..., -0.33333147,
        -1.48203703, -1.11832807],
       [-0.71481817, -0.66701875, -1.49005591, ..., -0.04772053,
        -1.04184432, -1.41742738],
       ...,
       [-0.71423555, -0.66737097, -1.8323224 , ...,  0.80911229,
        -1.26194067, -1.71652668],
       [ 0.95147513,  1.91328711, -0.05937724, ...,  1.66594512,
         0.05863748, -0.07148051],
       [ 1.49594387,  0.06724351,  0.48086085, ..., -0.33333147,
         1.04907109,  0.2276188 ]])

In [224]:
test = np.asanyarray(test)

In [225]:
result  =  model.predict(test)

In [226]:
result

array([[ 0.07624339],
       [-0.06124157],
       [-0.15887898],
       ...,
       [-0.29847451],
       [ 0.12828114],
       [ 0.05753357]])

In [227]:
output = pd.DataFrame(result)
output = output.rename(columns={0:"Snow Depth [cm]"})

In [228]:
output.index.names = ['Id']

In [233]:
output.to_csv('submission.csv')

## Accuracy

In [230]:
# mse
print("Mean_squared_error : %.2f" % (mean_squared_error(y_test , test_pred)))

Mean_squared_error : 8.42


In [231]:
# mean absolute error
print("Mean_absolute_error : %.2f" % (mean_absolute_error(y_test , test_pred)))

Mean_absolute_error : 0.23


In [232]:
# r squared error 
print('R^2 test: %.3f' % (r2_score(y_test , test_pred)))

R^2 test: 0.013
